Previous training with the initial 7 features gives high importance to the features mu, RA and t. So next step we can create new features using operations * and / with these features as component, that is, we create new features $mu\cdot RA, mu\cdot t, RA\cdot t, mu/RA, RA/t, mu/t$, add them into the training set and retrain the model.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance

2025-05-17 18:22:39.044512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747531359.172685 2298788 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747531359.206946 2298788 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747531359.474935 2298788 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747531359.474955 2298788 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747531359.474958 2298788 computation_placer.cc:177] computation placer alr

In [2]:
# Load data
data = pd.read_excel("data_gp.xlsx")

# Select input features and target
features = ['t', 'mu', 'RA', 'XA', 'XB', 'QA', 'Nd', 'mu*RA', 'mu*t', 'RA*t', 'mu/RA', 'RA/t', 'mu/t']
target = 'VRHE'
X = data[features].values
y = data[target].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train = X_scaled[:18, :]
y_train = y[:18]
X_val = X_scaled[18:, :]
y_val = y[18:]

In [3]:
X_train, y_train

(array([[-1.08223085,  1.62676953, -0.77345319,  0.86166497, -3.46038823,
          0.97887708, -1.88632043,  0.6919263 ,  1.19795976, -0.89069789,
          1.30696623, -0.03473135,  1.48595843],
        [-0.95938445,  1.18516573, -0.77345319,  0.86166497, -1.30350668,
          0.97887708,  0.04477912,  0.2266992 ,  0.62698919, -0.84320669,
          1.07547261, -0.28508627,  1.16041762],
        [-0.83653805,  0.79876242, -0.77345319,  0.86166497,  0.85337488,
          0.97887708,  1.97587866, -0.18037452,  0.14635215, -0.79571548,
          0.8729157 , -0.53294513,  0.83487681],
        [-1.20507726,  2.01317285, -0.77345319,  0.86166497, -1.36342005,
          0.97887708,  0.68847897,  1.09900001,  1.66440573, -0.9381891 ,
          1.50952314,  0.21815753,  1.77894515],
        [-0.81196877,  0.74356194, -0.77345319,  0.86166497,  0.75751347,
          0.97887708,  1.71839873, -0.2385279 ,  0.08870702, -0.78621724,
          0.84397899, -0.58222066,  0.83487681],
        [-0.811

In [4]:
X_val, y_val

(array([[ 0.66066848, -0.1314678 ,  0.84433277, -0.18925382, -1.30350668,
         -0.23810524, -0.59892073,  1.05285105,  0.86667197,  0.74906064,
         -0.61991747,  1.09160212, -0.42753331],
        [ 1.39284044, -0.17235704,  1.81500435, -0.81980509, -0.4946761 ,
         -0.96829463, -0.98514064,  2.33111699,  1.94859543,  1.65498368,
         -1.16162099,  2.30139937, -0.78150648],
        [ 1.2612462 , -1.17414342,  1.16788996, -0.39943758,  0.85337488,
         -0.4815017 ,  1.20343885,  0.27104274, -0.1928533 ,  1.20106435,
         -1.29060684,  0.81560111, -1.26611789],
        [ 0.82764462, -1.9714836 ,  0.2221074 , -1.17237139,  0.76350481,
         -1.45508756,  0.36662904, -1.85085908, -2.36157985,  0.44588185,
         -1.16575237, -0.93650746, -1.51830759],
        [ 2.21775967, -2.4825991 ,  1.80878209, -1.7825823 ,  0.85337488,
         -1.45508756,  0.68847897, -0.48254408, -1.36302611,  2.01872653,
         -2.20917896,  0.72733736, -2.34237777]]),
 array([1.672

Use the same 4-layer model.

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.06), input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.06)),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.06)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=480, verbose=1)

Epoch 1/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 11.1985 - mae: 1.7999
Epoch 2/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 10.7035 - mae: 1.6733
Epoch 3/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 10.2828 - mae: 1.5579
Epoch 4/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 9.9110 - mae: 1.4491
Epoch 5/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 9.5667 - mae: 1.3398
Epoch 6/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 9.2479 - mae: 1.2303
Epoch 7/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.9597 - mae: 1.1225
Epoch 8/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 8.7017 - mae: 1.0175
Epoch 9/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 8.4672 - mae: 0.9185
Epoch 10/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.2570 - mae: 0.8344
Epoch 11/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.0683 - mae: 0.7495
Epoch 12/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 7.9012 - mae: 0.6653
Epoch 13/480
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/st

In [15]:
# Evaluate model
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation MAE from model.evaluate: {val_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step - loss: 0.0642 - mae: 0.0257
Validation MAE from model.evaluate: 0.0257


The model is trained to have small overfitting.

In [16]:
r = permutation_importance(model, X_val, y_val,
                           scoring='r2',
                           n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    print(f"{features[i]:<8}"
            f"{r.importances_mean[i]:.3f}"
            f" +/- {r.importances_std[i]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━

The new features mu/t and mu/RA make great contribution to the model performance. Therefore, they cen be added to the feature pool for gplearn analysis to generate more specific mathematical formula to map these features to the OER activities. The result obtained by such neural network study aligns with the results obtained only by gplearn as demonstrated in the paper: Weng, B., Song, Z., Zhu, R. et al. Simple descriptor derived from symbolic regression accelerating the discovery of new perovskite catalysts. Nat Commun 11, 3513 (2020).